In [ ]:
!pip install influxdb
!pip install neuralprophet[live]

In [ ]:
from influxdb import DataFrameClient, InfluxDBClient
import pandas as pd
import numpy as np

In [ ]:
#Libreria influxdb

def getInfluxConnection(dbhost, dbport, dbuser, dbpasswd, dbname):
  return DataFrameClient(dbhost, dbport, dbuser, dbpasswd, dbname)

# Devuelve un diccionario con clave nombre de la estacion y valor el df entero
def getEstacion(estacion, cliente):
  return cliente.query(f'select * from {estacion}')[f'{estacion}']


In [ ]:
dbhost = '34.140.25.147' # Cuidado! Google Cloud puede cambiar la ip pública!
dbport = 8086
dbuser = 'root'
dbpasswd = '12345678'
dbname = 'registros_contaminacion'
cliente = getInfluxConnection(dbhost, dbport, dbuser, dbpasswd, dbname)

datos = getEstacion("ESTACION_8", cliente)

In [ ]:
datos

,BEN,CH4,CO,EBE,NMHC,NO,NO2,NOx,O3,PM10,PM25,PUNTO_MUESTREO,SO2,TCH,TOL
2001-01-01 00:00:00+00:00,4.0,NaN,0.0,2.0,NaN,66.0,67.0,168.0,7.0,32.0,NaN,28079008_1_38,26.0,NaN,11.0
2001-01-01 01:00:00+00:00,9.0,NaN,0.0,5.0,NaN,146.0,71.0,294.0,7.0,41.0,NaN,28079008_1_38,21.0,NaN,21.0
2001-01-01 02:00:00+00:00,9.0,NaN,0.0,5.0,NaN,190.0,73.0,364.0,7.0,50.0,NaN,28079008_1_38,22.0,NaN,24.0
2001-01-01 03:00:00+00:00,10.0,NaN,0.0,5.0,NaN,170.0,75.0,335.0,7.0,55.0,NaN,28079008_1_38,19.0,NaN,25.0
2001-01-01 04:00:00+00:00,8.0,NaN,0.0,4.0,NaN,102.0,67.0,224.0,8.0,42.0,NaN,28079008_1_38,14.0,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 19:00:00+00:00,NaN,NaN,0.0,NaN,NaN,9.0,54.0,68.0,40.0,14.0,5.0,28079008_1_38,3.0,NaN,NaN
2022-02-28 20:00:00+00:00,0.0,NaN,0.0,0.0,NaN,16.0,72.0,96.0,21.0,16.0,9.0,28079008_1_38,3.0,NaN,3.0
2022-02-28 21:00:00+00:00,0.0,NaN,0.0,0.0,NaN,14.0,77.0,99.0,16.0,17.0,9.0,28079008_1_38,3.0,NaN,3.0
2022-02-28 22:00:00+00:00,0.0,NaN,0.0,0.0,NaN,15.0,74.0,97.0,17.0,16.0,10.0,28079008_1_38,3.0,NaN,3.0


In [ ]:
df_aux = pd.DataFrame()

#df_aux['PM25'] = datos['PM25']
df_aux['PM10'] = datos['PM10']
#df_aux['NO2'] = datos['NO2']
#df_aux['O3'] = datos['O3']
#df_aux['SO2'] = datos['SO2']

#df_aux['CO'] = datos['CO']
#df_aux['NO'] = datos['NO']
#df_aux['NOx'] = datos['NOx']

prophet_df = df_aux.reset_index().rename(columns={'index':'ds', 'PM10':'y'})
prophet_df['ds']= prophet_df['ds'].dt.tz_localize(None)

# df_aux.isnull().sum()
#prophet_df = prophet_df.fillna(prophet_df.mean())

prophet_df = prophet_df.dropna()

prophet_df

,ds,y
0,2001-01-01 00:00:00,32.0
1,2001-01-01 01:00:00,41.0
2,2001-01-01 02:00:00,50.0
3,2001-01-01 03:00:00,55.0
4,2001-01-01 04:00:00,42.0
...,...,...
171874,2022-02-28 19:00:00,14.0
171875,2022-02-28 20:00:00,16.0
171876,2022-02-28 21:00:00,17.0
171877,2022-02-28 22:00:00,16.0


In [ ]:
prophet_df.isnull().sum()

ds    0
y     0
dtype: int64

In [ ]:
from neuralprophet import NeuralProphet #, set_log_level

m = NeuralProphet(
    
    #yearly_seasonality=False,
    #weekly_seasonality=False,
    #daily_seasonality=False,
    num_hidden_layers=4,
    d_hidden=16,
    learning_rate= 0.003,

    batch_size=64,
    epochs=200
    
    )

df_train, df_test = m.split_df(df=prophet_df, freq="h", valid_p=0.2)

metrics = m.fit(df=df_train, freq="h", validation_df=df_test)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.633% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - h
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.768% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - h
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
Epoch[200/200]: 100%|██████████| 200/200 [17:41<00:00,  5.31s/it, SmoothL1Loss=0.0423, MAE=15, RMSE=21.7, RegLoss=0, MAE_val=11.8, RMSE_val=16.1, SmoothL1Loss_val=0.0261]


In [ ]:
metrics

,SmoothL1Loss,MAE,RMSE,RegLoss,SmoothL1Loss_val,MAE_val,RMSE_val
0,0.752123,84.822804,101.553111,0.0,0.115593,33.025362,35.062066
1,0.087353,22.403487,31.438275,0.0,0.022513,10.971636,15.116075
2,0.047547,15.777239,23.255144,0.0,0.028862,12.585157,16.970095
3,0.045018,15.583745,22.596749,0.0,0.027503,12.209494,16.533848
4,0.045614,15.778260,22.711437,0.0,0.030504,13.352904,17.708020
5,0.046747,16.087812,23.017511,0.0,0.029873,13.287953,17.528427
6,0.048267,16.461416,23.374698,0.0,0.055572,19.556552,24.091812
7,0.050118,16.880386,23.810107,0.0,0.042146,16.122845,20.712732
8,0.051536,17.206119,24.151224,0.0,0.033247,13.939490,18.382609
9,0.053541,17.638867,24.604214,0.0,0.043382,16.655213,21.240751
